<a href="https://colab.research.google.com/github/tabba98/football/blob/main/PremierLeaguePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import json
import re
import requests
import pandas as pd

seasons = [2017, 2018, 2019, 2020, 2021, 2022]
competitions = ['EPL']

all_data = []
for season in seasons:
  for comp in competitions:
    url = f'https://understat.com/league/{comp}/{season}'
    html_doc = requests.get(url).text

    data = re.search(r"datesData\s*=\s*JSON\.parse\('(.*?)'\)", html_doc).group(1)
    data = re.sub(r'\\x([\dA-F]{2})', lambda g: chr(int(g.group(1), 16)), data)
    data =json.loads(data)

    for d in data:
      all_data.append({
          'season': season,
          'competitio': comp,
          'date': d['datetime'][:10],
          'home_team': d['h']['title'],
          'away_team': d['a']['title'],
          'home_goals': d['goals']['h'],
          'away_goals': d['goals']['a'],
          'home_xG': d['xG']['h'],
          'away_xG': d['xG']['a'],
          'forecast': list(d.get('forecast', {}).values())
      })

df = pd.DataFrame(all_data)

df[['home_win_prob', 'draw_prob', 'away_win_prob']] = df['forecast'].apply(lambda x: pd.Series(x))

df = df.drop('forecast', axis=1)

df = df.dropna(how='any', subset=None)
